In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM
import matplotlib.pyplot as plt

In [2]:
#Function to calculate date based on the given number
def calculate_date(row):
    base_date = datetime(2023, 3, 1)  # March 1st, 2023
    delta = timedelta(days=row['week'] - 1)
    return base_date + delta

In [3]:
def generar_predicciones(model,train_MRNN_scN,test_MRNN_scN,Y_scaler,n_input,n_features):
    test_predictions = []
    test_variables=test_MRNN_scN
    temporal=[]

    first_eval_batch = train_MRNN_scN[-n_input:]
    current_batch = first_eval_batch.reshape((1, n_input, n_features))
    test_batch= test_variables.reshape((1,len(test_variables),n_features))

    for i in range(len(test_MRNN_scN)):
        # get the prediction value for the first batch
        current_pred = model.predict(current_batch)[0]
    
        # append the prediction into the array
        test_predictions.append(current_pred[n_input-1,0])

        test_batch[0,i,0]=current_pred[n_input-1,0]
        #test_batch[0,i,0]=current_pred
        temporal= test_batch[0,i,:]
        temporal= temporal.reshape((1,1,6))

        # use the prediction to update the batch and remove the first value
        #current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)
        current_batch = np.append(current_batch[:,1:,:],temporal,axis=1)
    
    true_predictions = Y_scaler.inverse_transform([test_predictions])
    Predicciones=[]

    for i in range(len(test_MRNN_scN)):
        Predicciones.append(true_predictions[0][i])

    return Predicciones

In [6]:
df_multivariable = pd.read_csv('data/train.csv')
meals_list = [1878, 1754, 1558, 2581, 1962, 2104, 1571, 2444]
df_multivariable = df_multivariable[df_multivariable['meal_id'].isin(meals_list)]

In [7]:
df_multivariable['date'] = df_multivariable.apply(calculate_date, axis=1)
df_multivariable.set_index('date',inplace=True)
df_real_data = df_multivariable.copy(deep=True)

In [8]:
X_scaler = MinMaxScaler()
Y_scaler = MinMaxScaler()
X_data=X_scaler.fit_transform(df_multivariable.drop(columns=['num_orders']))
Y_data = Y_scaler.fit_transform(df_multivariable[['num_orders']])

In [9]:
df_multivariable[['id', 'week',	'center_id', 'meal_id', 'checkout_price', 'base_price', 'emailer_for_promotion', 'homepage_featured']] = X_data
df_multivariable[['num_orders']] = Y_data

In [10]:
df_multivariable

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
date,,,,,,,,,
2023-03-01,0.481874,0.0,0.255682,0.191593,0.301761,0.168721,0.0,0.0,0.038330
2023-03-01,0.676248,0.0,0.255682,0.000000,0.730902,0.726737,1.0,0.0,0.012100
2023-03-01,0.376745,0.0,0.255682,1.000000,0.730902,0.730214,1.0,0.0,0.024281
2023-03-01,0.880032,0.0,0.255682,0.394917,0.729442,0.730214,1.0,0.0,0.017703
2023-03-01,0.484377,0.0,0.255682,0.866080,0.796008,0.755402,0.0,0.0,0.000162
...,...,...,...,...,...,...,...,...,...
2023-07-23,0.576931,1.0,0.289773,0.312805,0.347039,0.224364,0.0,0.0,0.008933
2023-07-23,0.788392,1.0,0.289773,0.191593,0.351332,0.225998,0.0,0.0,0.042797
2023-07-23,0.759437,1.0,0.289773,0.000000,0.679871,0.618875,0.0,0.0,0.012100


In [11]:
train_MRNN_sc=df_multivariable['2023-03-01':'2023-06-30']
test_MRNN_sc=df_multivariable['2023-07-01':]

In [12]:
train_MRNN_scN=train_MRNN_sc.to_numpy()
test_MRNN_scN=test_MRNN_sc.to_numpy()
n_input = int(7)
n_features = 9
generator = TimeseriesGenerator(train_MRNN_scN, train_MRNN_scN, length=n_input, batch_size=1)   

In [13]:
# define model
model = Sequential()
model.add(LSTM(64, return_sequences=True,activation='relu', input_shape=(n_input, n_features)))
model.add(LSTM(128, return_sequences=True,activation='relu'))
model.add(LSTM(256, return_sequences=True,activation='relu'))
model.add(LSTM(128, return_sequences=True,activation='relu'))
model.add(LSTM(64, return_sequences=True,activation='relu'))
#model.add(Dense(1,activation='linear'))
model.add(LSTM(n_features, return_sequences=True)) 

In [15]:
model.compile(optimizer='adam', loss='mse')
history=model.fit(generator,epochs=20)

Epoch 1/20


11844/58517 [=====>........................] - ETA: 4:54 - loss: 0.0523

KeyboardInterrupt: 

In [ ]:
mse = history.history['loss']

epochs=range(len(mse)) # Get number of epochs
#------------------------------------------------
# Plot MAE, MSE and Loss
#------------------------------------------------
plt.figure(figsize=(10, 6));
plt.plot(epochs, mse, 'green', label='MSE')
plt.title("Number of epochs vs MSE")
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.legend();

In [ ]:
predicciones=generar_predicciones(model,train_MRNN_scN,test_MRNN_scN,Y_scaler,n_input,n_features)

testinverse=pd.DataFrame(Y_scaler.inverse_transform(test_MRNN_sc), index=test_MRNN_sc.index,columns=test_MRNN_sc.columns)
testinverse['Predictions']=predicciones

mse = mean_squared_error(df_real_data['BMW']['2021-07-02':'2021-12-31'], testinverse['Predictions'])

In [ ]:
df_test = pd.read_csv('data/test.csv')
# Apply the function to create the 'Date' column
df_test['date'] = df_test.apply(calculate_date, axis=1)